In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

In [2]:
#Start session Context com nome ENEM
spark = SparkSession.builder.appName("ENEM").getOrCreate()

In [3]:
%%time
enem = (
    spark
    .read
    .format("csv")
    .options(header=True, sep=";", encoding="latin1", inferSchema=True)
    .load("MICRODADOS_ENEM_2020.csv")
 )


Wall time: 18.4 s


In [4]:
enem.printSchema()

root
 |-- NU_INSCRICAO: long (nullable = true)
 |-- NU_ANO: integer (nullable = true)
 |-- TP_FAIXA_ETARIA: integer (nullable = true)
 |-- TP_SEXO: string (nullable = true)
 |-- TP_ESTADO_CIVIL: integer (nullable = true)
 |-- TP_COR_RACA: integer (nullable = true)
 |-- TP_NACIONALIDADE: integer (nullable = true)
 |-- TP_ST_CONCLUSAO: integer (nullable = true)
 |-- TP_ANO_CONCLUIU: integer (nullable = true)
 |-- TP_ESCOLA: integer (nullable = true)
 |-- TP_ENSINO: integer (nullable = true)
 |-- IN_TREINEIRO: integer (nullable = true)
 |-- CO_MUNICIPIO_ESC: integer (nullable = true)
 |-- NO_MUNICIPIO_ESC: string (nullable = true)
 |-- CO_UF_ESC: integer (nullable = true)
 |-- SG_UF_ESC: string (nullable = true)
 |-- TP_DEPENDENCIA_ADM_ESC: integer (nullable = true)
 |-- TP_LOCALIZACAO_ESC: integer (nullable = true)
 |-- TP_SIT_FUNC_ESC: integer (nullable = true)
 |-- CO_MUNICIPIO_PROVA: integer (nullable = true)
 |-- NO_MUNICIPIO_PROVA: string (nullable = true)
 |-- CO_UF_PROVA: integer 

In [7]:
enem.createOrReplaceTempView('enem')

In [13]:
%%time
#quantidade de registros
spark.sql("""
    select count(*) as quantidade from enem
""").show()

+----------+
|quantidade|
+----------+
|   5783109|
+----------+

Wall time: 2.24 s


In [18]:
%%time
#Qual é a média da nota em matemática dos alunos que estudaram numa escola em Minas Gerais?
spark.sql("""
    select round(avg(NU_NOTA_MT),2) as media from enem
    where SG_UF_ESC = 'MG'
""").show()



+------+
| media|
+------+
|579.68|
+------+

Wall time: 4.36 s


In [21]:
spark.sql("""
 select round(avg(NU_NOTA_MT),2) as media from enem
    where SG_UF_ESC = 'MG'
""").explain()
            

== Physical Plan ==
*(2) HashAggregate(keys=[], functions=[avg(NU_NOTA_MT#54)])
+- Exchange SinglePartition, ENSURE_REQUIREMENTS, [id=#165]
   +- *(1) HashAggregate(keys=[], functions=[partial_avg(NU_NOTA_MT#54)])
      +- *(1) Project [NU_NOTA_MT#54]
         +- *(1) Filter (isnotnull(SG_UF_ESC#35) AND (SG_UF_ESC#35 = MG))
            +- FileScan csv [SG_UF_ESC#35,NU_NOTA_MT#54] Batched: false, DataFilters: [isnotnull(SG_UF_ESC#35), (SG_UF_ESC#35 = MG)], Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/014207631/MICRODADOS_ENEM_2020.csv], PartitionFilters: [], PushedFilters: [IsNotNull(SG_UF_ESC), EqualTo(SG_UF_ESC,MG)], ReadSchema: struct<SG_UF_ESC:string,NU_NOTA_MT:double>




In [27]:
%%time
#Qual é a média da nota em Linguagens e Códigos dos alunos que estudaram numa escola em Minas Gerais?

consulta2 = (enem
             .filter(enem.SG_UF_ESC == 'MG')
             .agg(
                 f.mean('NU_NOTA_LC').alias('MEDIA NOTA')
                  )
                 
            ) 
consulta2.show()

+-----------------+
|       MEDIA NOTA|
+-----------------+
|553.0102913021103|
+-----------------+

Wall time: 3.98 s


In [28]:
%%time
#Qual é a média da nota em Ciências Humanas dos alunos do sexo FEMININO que estudaram numa escola em Minas Gerais?
consulta3 = (enem
             .filter((enem.TP_SEXO == 'F') & (enem.SG_UF_ESC == 'MG'))
             .agg( f.mean('NU_NOTA_CH').alias('MÉDIA'))
            )
consulta3.show()

+-----------------+
|            MÉDIA|
+-----------------+
|541.3084611723356|
+-----------------+

Wall time: 3.93 s


In [29]:
%%time
#Qual é a média da nota em Ciências Humanas dos alunos do sexo MASCULINO que estudaram numa escola em Minas Gerais?
consulta4 = (enem
             .filter((enem.TP_SEXO == 'M') & (enem.SG_UF_ESC == 'MG'))
             .agg( f.mean('NU_NOTA_CH').alias('MÉDIA NOTA'))
            ) 
consulta4.show()

+-----------------+
|       MÉDIA NOTA|
+-----------------+
|559.5546844135076|
+-----------------+

Wall time: 4 s


In [30]:
%%time
#Qual é a média da nota em Matemática dos alunos do sexo FEMININO que estudaram numa escola na cidade de Montes Claros?
consulta5 = (enem
             .filter((enem.TP_SEXO == 'F')&(enem.NO_MUNICIPIO_ESC == 'Montes Claros'))
             .agg( f.round(f.mean('NU_NOTA_MT'),2).alias('MEDIA'))
            ) 
consulta5.show()

+------+
| MEDIA|
+------+
|562.99|
+------+

Wall time: 4.12 s


In [31]:
%%time
#Qual é a média da nota em Matemática dos alunos que estudaram numa escola no município de Sabará e que possuem TV por assinatura na residência?
consulta6 = (enem
             .filter((enem.NO_MUNICIPIO_ESC == 'Sabará')&(enem.Q021=='B'))
             .agg( f.round(f.mean('NU_NOTA_MT'),2).alias('MEDIA'))
                 )      
consulta6.show()

+-----+
|MEDIA|
+-----+
|580.1|
+-----+

Wall time: 4.48 s


In [35]:
%%time
#Qual é a média da nota em Ciências Humanas dos alunos que estudaram numa escola em Minas Gerais e que possuem dois fornos micro-ondas em casa?
consulta7= (enem
       .filter((enem.Q016 == 'C')&(enem.SG_UF_ESC == 'MG'))
       .agg(  f.round(f.mean('NU_NOTA_CH'),2)).alias('MEDIA')
      ) 
consulta7.show()

+-------------------------+
|round(avg(NU_NOTA_CH), 2)|
+-------------------------+
|                   601.48|
+-------------------------+

Wall time: 3.89 s


In [39]:
%%time
#Qual é a nota média em Matemática dos alunos que estudaram numa escola em Minas Gerais e cuja mãe completou a pós-graduação?
consulta8 = (enem
             .filter((enem.Q002 == 'G')&(enem.SG_UF_ESC == 'MG'))
             .agg( f.round(f.mean('NU_NOTA_MT'),2).alias('MEDIA'))
             )
consulta8.show()

+------+
| MEDIA|
+------+
|642.25|
+------+

Wall time: 3.88 s


In [42]:
%%time
#Qual é a nota média em Matemática dos alunos que estudaram numa escola em Belo Horizonte ou em Conselheiro Lafaiete?
consulta9 = (enem
             .filter((enem.NO_MUNICIPIO_ESC == 'Conselheiro Lafaiete')|
                     (enem.NO_MUNICIPIO_ESC == 'Belo Horizonte'))
             .agg( f.round(f.mean('NU_NOTA_MT'),2)
                    .alias('media'))
                 )     
consulta9.show()

+------+
| media|
+------+
|621.03|
+------+

Wall time: 3.99 s
